In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from datetime import datetime
import datetime as dt
import time

#Read the data - SAP, SFDC & Ecomm
SAP = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\01_SAP_Raw Report_IQ.xlsx")

SFDC = pd.read_csv("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\SFDC_Raw Report.csv", parse_dates=['Date/Time Opened'], dayfirst=True, encoding = "cp1252")

Ecomm = pd.read_csv("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\ECOMM Raw Data.csv")

#Remove duplicates on SAP & Ecomm data
SAP = SAP.drop_duplicates(subset=['Sales Document'])
Ecomm = Ecomm.drop_duplicates(subset=['TELUS_ORDER_ID'])

#Remove false time from Document Date, Delivery Date & Goods Issue Date
SAP['Goods Issue Date'] = SAP['Goods Issue Date'].dt.strftime('%d/%m/%Y')
SAP['Document Date'] = SAP['Document Date'].dt.strftime('%d/%m/%Y')
SAP['Delivery Date'] = SAP['Delivery Date'].dt.strftime('%d/%m/%Y')

#Adjust SFDC date/time
SFDC['Date/Time Opened'] = (SFDC['Date/Time Opened']).dt.strftime('%Y-%m-%d %H:%M:%S')

#Rename some columns
SFDC.rename(columns = {'Case Number' : 'SFDC Case #', 'Date/Time Opened' : 'Received in SFDC'}, inplace = True)
Ecomm.rename(columns = {'SUBMITTED_BY' : 'Submitter'}, inplace = True)


In [2]:
#Merge all three data

R1 = pd.merge(left=SAP, right=Ecomm, left_on='Reference', right_on='TELUS_ORDER_ID', how='left')
#R1.head()
# R2 = R1.join(SFDC, how='inner', lsuffix='_left', rsuffix='_right')
SFDC = SFDC.drop_duplicates(subset=['IQ TL #'])
R2 = pd.merge(left=R1, right=SFDC, left_on='Reference', right_on='IQ TL #', how='left')
#R2

R3 = R2[['Plant', 'Sales Group', 'Sold-to party', 'Ship-to party', 'Ship-to Province', 'Sales Document', 'Document Date', 'Submitter', 'Delivery', 'Delivery Date', 'Goods Issue Date', 'Reference', 
                 'SFDC Case #', 'Case Owner', 'Received in SFDC']]


In [3]:
#Check for backorders

sf = pd.read_csv("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\SFDC_Raw Report.csv", encoding = "cp1252")
BO = sf[(sf['Old Value'] == 'Backorder') | (sf['New Value'] == 'Backorder')]

#Rename column to reflect BO
BO.rename(columns = {'Case Number' : 'Backorder?'}, inplace = True)
BO = BO[['Backorder?']]
BO = BO.drop_duplicates()


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [4]:
#Add backorder to table

R4 = R3.merge(BO, left_on='SFDC Case #', right_on='Backorder?', how='left')


In [5]:
#Create 2 reference tables from clientwikki and combine them
Clientwiki = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\01 - WSS Corporate Client List.xlsx", sheet_name = 'Master Client List', engine = 'openpyxl')

ck1 = Clientwiki[['SAP Account', 'Current Delivery Method', 'Team']]
ck2 = Clientwiki[['BAN', 'Current Delivery Method', 'Team']]

#Rename Sap Account & BAN
ck1.rename(columns = {'SAP Account' : 'Account'}, inplace = True)
ck2.rename(columns = {'BAN' : 'Account'}, inplace = True)

#Combine both
ck3 = pd.concat([ck1, ck2])


In [6]:
#Create reference table from clientwikki
Ref = pd.merge(left=Ecomm, right=ck3, left_on='ACCOUNT', right_on='Account', how='left')

#Rename some columns to align with output
Ref.rename(columns = {'TELUS_ORDER_ID' : 'TL', 'Team' : 'City', 'Current Delivery Method' : 'Flow'}, inplace = True)

#Add a column to specify whether WSS or not
Ref['WSS?'] = np.where(Ref['City'].isnull(), 'Non WSS', 'IQ WSS')

#Final reference table - remove duplicate on the TL column
Ref = Ref[['TL', 'WSS?', 'City', 'Flow']]
Ref = Ref.drop_duplicates(subset=['TL'])
#Ref.head()

#Add reference table to output
R5 = pd.merge(left=R4, right=Ref, left_on='Reference', right_on='TL', how='left')


In [7]:
#Read SAP again so as to concatenate date & time

sp = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\01_SAP_Raw Report_IQ.xlsx")
sp['Doc Time'] = pd.to_datetime(sp['Document Date']) + pd.to_timedelta(sp['Document Time'].astype(str))
sp['Del Time'] = pd.to_datetime(sp['Delivery Date']) + pd.to_timedelta(sp['Delivery Time'].astype(str))
sp['Gd Time'] = pd.to_datetime(sp['Goods Issue Date']) + pd.to_timedelta(sp['Goods Issue Time'].astype(str))
sp_new = sp[['Sales Document', 'Doc Time', 'Del Time', 'Gd Time']]

#Remove duplicate on sales document
sp_new = sp_new.drop_duplicates(subset=['Sales Document'])

In [8]:
#Add SFDC recieved time to the sp table by joining sp to R5 using sale document column

R6 = pd.merge(left=R5, right=sp_new, on='Sales Document', how='left')


In [1]:
#Calculate date without weekends and holidays

COL1 = "SFDC_Goods (Mins)"
COL2 = "SFDC_SAP (Mins)"
COL3 = "SAP_WH (Mins)"
COL4 = "WH_Goods (Mins)"
COL5 = "SAP_Goods (Mins)"

hd = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\Holidays.xlsx")
HOLIDAYS = hd['HolidayDate']

def get_diff(date1, date2):
    try:
        diff = abs((date2 - date1).total_seconds())
        days_between = (date2.date() - date1.date()).days
        weekdays_between = abs(np.busday_count(date2.date(), date1.date()))
        weekends_between = days_between - weekdays_between

        if date2.date() != date1.date():
            diff = diff - (weekends_between * SECONDS_IN_A_DAY)
            
        for holiday in HOLIDAYS:
            holiday_date = datetime.strptime(str(holiday), "%Y-%m-%d %H:%M:%S")
            if date1.date() <= holiday_date.date() <= date2.date():
                diff = diff -  SECONDS_IN_A_DAY


        return int(diff / 60)

    except Exception as e:
        print(e)
        return 0


SECONDS_IN_A_DAY = 86400
R6[COL1] = 0
R6[COL2] = 0
R6[COL3] = 0
R6[COL4] = 0
R6[COL5] = 0

for index, row in R6.iterrows():
    try:
        sfdc_date = datetime.strptime(str(row["Received in SFDC"]), "%Y-%m-%d %H:%M:%S")
        doc_time_date = datetime.strptime(str(row["Doc Time"]), "%Y-%m-%d %H:%M:%S")
        del_time_date = datetime.strptime(str(row["Del Time"]), "%Y-%m-%d %H:%M:%S")
        gd_time_date = datetime.strptime(str(row["Gd Time"]), "%Y-%m-%d %H:%M:%S")
   
        R6.at[index, COL1] = get_diff(sfdc_date, gd_time_date)
        R6.at[index, COL2] = ifor_val = get_diff(sfdc_date, doc_time_date)
        R6.at[index, COL3] = get_diff(doc_time_date, del_time_date)
        R6.at[index, COL4] = get_diff(del_time_date, gd_time_date)
        R6.at[index, COL5] = get_diff(doc_time_date, gd_time_date)
    except Exception as e:
        print(e)
        


IndentationError: expected an indented block (Temp/ipykernel_10788/3365823358.py, line 56)

In [10]:
#Export IQ Result

IQ = R6[['Plant', 'Sales Group', 'Sold-to party', 'Ship-to party', 'Ship-to Province', 'Sales Document', 'Document Date', 
         'Submitter', 'Delivery', 'Delivery Date', 'Goods Issue Date', 'Reference', 'WSS?', 'City', 'Flow', 'SFDC Case #', 
         'Case Owner', 'Received in SFDC', 'Goods Issue Date', 'SFDC_Goods (Mins)', 'SFDC_SAP (Mins)', 'SAP_WH (Mins)', 
         'WH_Goods (Mins)', 'SAP_Goods (Mins)', 'Backorder?']]
export_excell = IQ.to_excel(r'C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\IQ_Output.xlsx', index=None, header=True)